In [1]:
from random import random, expovariate
from matplotlib import pyplot as plt

In [2]:
n = 1  # número de guichês
lamb = 0.2  # taxa de entrada de clientes
mi = 10  # taxa de atendimentos a clientes por cada guichê
t = 1000  # intervalo de tempo total sobre o qual se deseja calcular as médias de aceitações e rejeições
th = 10  # tempo que cada guichê demora para atender um cliente
rodadas = 1000  # número de iterações da simulação

In [3]:
class Cliente:
    
    tempo_atendimento = -1
    
    def __init__(self, momento_chegada):
        self.momento_chegada = momento_chegada
    
    def vai_desistir(self, tam_fila):
        if (tam_fila / (tam_fila + n)) > random():
            return True
        return False

In [4]:
def momentos_chegada(tempo_total, lamb):
    momentos = set()
    for i in range(tempo_total):
        momentos.add(int(expovariate(lamb)))
    return sorted(momentos)

In [5]:
def rodada():
    fila = []
    guiches_ocupados = []
    atendidos = []
    rejeicoes = 0
    chegadas = momentos_chegada(t, lamb)
    for momento in range(t):
        
        # Libere guichês que ficaram vazios desde a última iteração
        for guiche in guiches_ocupados:
            if guiche.tempo_atendimento == momento:
                # Se o guichê foi liberado, aumente o contador de clientes atendidos
                guiches_ocupados.remove(guiche)
                atendidos.append(guiche)
        
        # Se há clientes na fila, coloque-os em guichês livres (se houver algum)
        for cliente in fila:
            if len(guiches_ocupados) < n:
                # Aponte um guichê para esse cliente
                # TODO: estabeleça o tempo correto de atendimento
                cliente.tempo_atendimento = momento + th
                guiches_ocupados.append(cliente)
                
        # Descubra se um cliente chegou agora
        if momento in chegadas:
            cliente = Cliente(momento)
            # Calcule se o cliente irá permanecer na loja
            if cliente.vai_desistir(len(fila)):
                # Se não for, aumente o contador de rejeições
                rejeicoes += 1
            else:
                # Se ele for o único na fila, veja se ele pode ser atendido agora
                if(len(guiches_ocupados) < n):
                    # TODO: estabeleça o tempo correto de atendimento
                    cliente.tempo_atendimento = momento + th
                    guiches_ocupados.append(cliente)
                else:
                    # Se não houver, coloque-o na fila
                    fila.append(cliente)

    return {
        'fila': fila,
        'giches_ocupados': guiches_ocupados,
        'atendidos': atendidos,
        'rejeicoes': rejeicoes
    }

In [6]:
iteracoes = []
for i in range(rodadas):
    iteracoes.append(rodada())